In [ ]:
!python -m pip install -q openai

In [ ]:
import os
import pandas as pd
import sqlite3
import json
from openai import OpenAI

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd "/content/drive/MyDrive/290_Xplore"
db_path  = "mydb.sqlite"

/content/drive/MyDrive/290_Xplore


In [ ]:

conn = sqlite3.connect(db_path)
cur = conn.cursor()

In [ ]:
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
cur.fetchall()

[('restaurants',)]

In [ ]:
cur.execute("PRAGMA table_info(restaurants);")
cur.fetchall()

[(0, 'id', 'INTEGER', 0, None, 0),
 (1, 'position', 'INTEGER', 0, None, 0),
 (2, 'name', 'TEXT', 0, None, 0),
 (3, 'score', 'REAL', 0, None, 0),
 (4, 'ratings', 'REAL', 0, None, 0),
 (5, 'category', 'TEXT', 0, None, 0),
 (6, 'price_range', 'TEXT', 0, None, 0),
 (7, 'full_address', 'TEXT', 0, None, 0),
 (8, 'zip_code', 'TEXT', 0, None, 0),
 (9, 'lat', 'REAL', 0, None, 0),
 (10, 'lng', 'REAL', 0, None, 0)]

In [ ]:
cur.execute("SELECT * from restaurants limit 5");
cur.fetchall()

[(1,
  19,
  'PJ Fresh (224 Daniel Payne Drive)',
  None,
  None,
  'Burgers, American, Sandwiches',
  '$',
  '224 Daniel Payne Drive, Birmingham, AL, 35207',
  '35207',
  33.5623653,
  -86.8307025),
 (2,
  9,
  "J' ti`'z Smoothie-N-Coffee Bar",
  None,
  None,
  'Coffee and Tea, Breakfast and Brunch, Bubble Tea',
  None,
  '1521 Pinson Valley Parkway, Birmingham, AL, 35217',
  '35217',
  33.58364,
  -86.77333),
 (3,
  6,
  'Philly Fresh Cheesesteaks (541-B Graymont Ave)',
  None,
  None,
  'American, Cheesesteak, Sandwiches, Alcohol',
  '$',
  '541-B Graymont Ave, Birmingham, AL, 35204',
  '35204',
  33.5098,
  -86.85464),
 (4,
  17,
  "Papa Murphy's (1580 Montgomery Highway)",
  None,
  None,
  'Pizza',
  '$',
  '1580 Montgomery Highway, Hoover, AL, 35226',
  '35226',
  33.4044388,
  -86.8066142),
 (5,
  162,
  'Nelson Brothers Cafe (17th St N)',
  4.7,
  22.0,
  'Breakfast and Brunch, Burgers, Sandwiches',
  None,
  '314 17th St N, Birmingham, AL, 35203',
  '35203',
  33.51473,
  -8

In [ ]:
menus_df = pd.read_csv("restaurant-menus.csv")
menus_df.to_sql("menus", conn, if_exists="replace", index=False, chunksize=1000)

5117217

In [ ]:
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
cur.fetchall()

[('restaurants',), ('menus',)]

In [ ]:
cur.execute("PRAGMA table_info(menus);")
cur.fetchall()

[(0, 'restaurant_id', 'INTEGER', 0, None, 0),
 (1, 'category', 'TEXT', 0, None, 0),
 (2, 'name', 'TEXT', 0, None, 0),
 (3, 'description', 'TEXT', 0, None, 0),
 (4, 'price', 'TEXT', 0, None, 0)]

In [ ]:
cur.execute("SELECT * from menus limit 5");
cur.fetchall()

[]

In [ ]:
conn.close()

In [ ]:
input_7 = []
for idx, i in enumerate(a):
    name = i[2] if i[2] is not None else ""
    extra = i[3] if i[3] is not None else ""

    cur = (
        "restaurant id: 7; "
        "item_id: " + str(idx) + "; "
        "item_name: " + name + " " + extra
    )

    input_7.append(cur)

In [ ]:
items = []
for i in range(0, len(input_7), 10):
    input_cur = min(i + 10, len(input_7))
    response = get_chatgpt_response(str(input_7[i:input_cur]))
    data = json.loads(response.output_text)
    for item in data["results"]:
        items.append((item['item_name'], item['ingredients']))


In [ ]:
print(items[10])

('Rock Shrimp Tempura fried shrimp tossed in a yuzu aoli.', [{'name': 'rock shrimp', 'confidence': 0.95}, {'name': 'tempura batter', 'confidence': 0.9}, {'name': 'vegetable oil', 'confidence': 0.9}, {'name': 'mayonnaise', 'confidence': 0.88}, {'name': 'yuzu juice', 'confidence': 0.86}, {'name': 'salt', 'confidence': 0.7}, {'name': 'garlic', 'confidence': 0.6}, {'name': 'scallion', 'confidence': 0.4}])


In [ ]:
from openai import OpenAI
def get_chatgpt_response(menu_items):
  chatgpt_base_url = "https://aiapi-prod.stanford.edu/v1"
  chatgpt_model = "gpt-5"

  chatgpt_api_version = "2025-03-01-preview"

  chatgpt_client = OpenAI(
      api_key="sk-7nZuV2mGMDXZRxETx6rk6w",
      base_url=chatgpt_base_url,
      default_query={"api-version": chatgpt_api_version},
  )
  json_schema = {
      "format": {
          "type": "json_schema",
          "name": "results",
          "strict": True,
          "schema": {
              "type": "object",
              "additionalProperties": False,
              "properties": {
                  "results": {
                      "type": "array",
                      "items": {
                          "type": "object",
                          "additionalProperties": False,
                          "properties": {
                              "item_id": {"type": "string"},
                              "item_name": {"type": "string"},
                              "restaurant_id": {"type": "string"},
                              "ingredients": {
                                  "type": "array",
                                  "items": {
                                      "type": "object",
                                      "additionalProperties": False,
                                      "properties": {
                                          "name": {"type": "string"},
                                          "confidence": {
                                              "type": "number",
                                              "minimum": 0.0,
                                              "maximum": 1.0
                                          }
                                      },
                                      "required": ["name", "confidence"]
                                  }
                              },
                              "reasoning": {"type": "string"}
                          },
                          "required": [
                              "item_id",
                              "item_name",
                              "restaurant_id",
                              "ingredients",
                              "reasoning"
                          ]
                      }
                  }
              },
              "required": ["results"]
          }
      }
  }


  assistant_instruction = f'''
  You are a food menu analysis assistant.
  Task:
  Decompose each restaurant menu item into its likely ingredients.

  Input:
  You will be given a list of menu items. Each item contains:
  - item_id
  - item_name
  - restaurant_id
  - optional description

  Output requirements (STRICT):
  - Output MUST be machine-readable.
  - Output MUST follow the provided JSON schema exactly.
  - Do NOT include extra commentary, explanations, or markdown.
  - Do NOT invent fields that are not requested.
  - Each ingredient must be a short noun singular phrase.
  - Ingredients must be ordered from most prominent to least prominent.
  - Each ingredient must include a confidence score between 0 and 1.
  - The confidence represents how likely the ingredient is to be present in the dish.

  For each menu item, infer:
  1) The most likely ingredients commonly used in this dish
  2) A confidence score (0–1) for each ingredient
  3) The reasoning explaining WHY those ingredients are present, based on:
    - culinary conventions
    - dish name semantics
    - regional cooking practices
    - typical restaurant preparation methods

  Constraints:
  - If an ingredient is uncertain, still include it but give it a lower confidence.
  - Do NOT include cooking utensils or heat
  - Avoid overly granular items (e.g., “Himalayan pink salt” → “salt”).
  '''


  chatgpt_response = chatgpt_client.responses.create(
      model=chatgpt_model,
      input=menu_items,
      instructions=assistant_instruction,
      text=json_schema,
  )
  return chatgpt_response

# print(response.output_text)

In [ ]:
import json

data = json.loads(chatgpt_response.output_text)
print(type(data))
print(data["results"][1])  # individual item result

<class 'dict'>
{'item_id': '1', 'item_name': 'Edamame Steamed or grilled. Served with ponzu.', 'restaurant_id': '7', 'ingredients': [{'name': 'edamame', 'confidence': 0.99}, {'name': 'ponzu sauce', 'confidence': 0.9}, {'name': 'salt', 'confidence': 0.8}, {'name': 'vegetable oil', 'confidence': 0.4}], 'reasoning': 'Edamame is commonly steamed or grilled and lightly salted; the description states it’s served with ponzu. Grilled preparation often involves a light toss in oil, hence low-confidence vegetable oil.'}


In [ ]:
print(data["results"][1]['ingredients'])

[{'name': 'edamame', 'confidence': 0.99}, {'name': 'ponzu sauce', 'confidence': 0.9}, {'name': 'salt', 'confidence': 0.8}, {'name': 'vegetable oil', 'confidence': 0.4}]


In [ ]:
print(chatgpt_response)
chatgpt_data = json.loads(chatgpt_response.output_text)
print(type(chatgpt_data))
print(chatgpt_data["results"][1])  # individual item result

<class 'dict'>
{'item_id': '10', 'item_name': 'Jumbo Chicken Wings Five Pieces', 'restaurant_id': '1', 'ingredients': ['chicken wings', 'Buffalo wing sauce', 'blue cheese dressing', 'celery sticks', 'ranch dressing'], 'reasoning': 'Jumbo wings typically refer to large fried or baked chicken wings served sauced. In many US pizza-shop menus, the default or most common sauce is Buffalo wing sauce, and they are commonly accompanied by blue cheese or ranch dressing and celery sticks. Exact sauce and dip can vary by restaurant, so the accompaniments are somewhat uncertain.'}


In [ ]:
chatgpt_instruction_detailed = f'''
You are a food menu analysis assistant.

Goal:
Given menu items, infer ingredients as detailed as possible while staying realistic.

Input:
Each menu item contains:
- item_id
- item_name
- restaurant_id
- optional description

Output requirements (STRICT):
- Output MUST be valid JSON matching the provided JSON schema.
- Do NOT include any extra keys beyond the schema.
- Do NOT include any markdown or commentary.

Ingredient inference guidelines:
- Provide a comprehensive ingredient list for each item.
- Include likely sub-components where appropriate (e.g., crust/dough, sauce, cheese, protein, vegetables, toppings, fillings, garnish).
- Include likely condiments/sauces/dips and typical sides when the dish commonly comes with them (only if plausible).
- Prefer common, restaurant-standard ingredients over niche/brand-specific ones.
- Avoid tools/heat words; avoid generic placeholders like "seasoning" unless necessary.
- Do not overfit: if uncertain, still include plausible ingredients but explain uncertainty in the reasoning.

Reasoning requirements:
- 3-6 sentences.
- Explicitly justify major ingredients (proteins, starch base, key sauces, signature toppings).
- Mention common variations briefly when relevant.

Now produce the JSON for the input.
'''

chatgpt_response_detailed = chatgpt_client.responses.create(
    model=chatgpt_model,
    input=chatgpt_menu_items,
    instructions=chatgpt_instruction_detailed,
    text=chatgpt_json_schema,
)

chatgpt_data_detailed = json.loads(chatgpt_response_detailed.output_text)
print(chatgpt_data_detailed["results"][1])

{'item_id': '10', 'item_name': 'Jumbo Chicken Wings Five Pieces', 'restaurant_id': '1', 'ingredients': ['chicken wings (drumettes and flats)', 'salt', 'black pepper', 'garlic powder', 'paprika', 'canola oil', 'Buffalo wing sauce (hot sauce, butter, vinegar, salt)', 'celery sticks', 'carrot sticks', 'ranch dressing'], 'reasoning': 'Jumbo wings typically mean large, unbreaded chicken wings seasoned simply with salt, pepper, and a few common spices, then served in a set count. Buffalo sauce is a prevalent default for wings, so listing a hot sauce–butter–vinegar style sauce is realistic. Celery and sometimes carrot sticks plus a creamy dip are customary accompaniments; ranch is a common default, though blue cheese is also widely offered. Some venues serve them plain or with alternative sauces (BBQ, garlic parmesan), but Buffalo with ranch and vegetable sticks is a plausible standard.'}


In [ ]:
# Gemini (gemini-2.5-pro)
with open(os.path.join(current_dir, "API_Key"), "r") as f:
    gemini_api_key = f.read().strip()

gemini_base_url = "https://aiapi-prod.stanford.edu/v1"
gemini_model = "gemini-2.5-pro"

gemini_client = OpenAI(api_key=gemini_api_key, base_url=gemini_base_url)
gemini_menu_items = "1 item_id: 1; restaurant_id: 1; Extra Large Meat Lovers Whole Pie 2 item_id: 10; restaurant_id: 1; Jumbo Chicken Wings Five Pieces 3 item_id: 19; restaurant_id: 1; Bon Appetit Cheese and Berry Danish Single"
gemini_json_schema = {
    "format": {
        "type": "json_schema",
        "name": "results",
        "strict": True,
        "schema": {
            "type": "object",
            "additionalProperties": False,
            "properties": {
                "results": {
                    "type": "array",
                    "items": {
                        "type": "object",
                        "additionalProperties": False,
                        "properties": {
                            "item_id": {"type": "string"},
                            "item_name": {"type": "string"},
                            "restaurant_id": {"type": "string"},
                            "ingredients": {
                                "type": "array",
                                "items": {"type": "string"}
                            },
                            "reasoning": {"type": "string"},
                        },
                        "required": [
                            "item_id",
                            "item_name",
                            "restaurant_id",
                            "ingredients",
                            "reasoning"
                        ]
                    }
                }
            },
            "required": ["results"]
        }
    }
}

gemini_instruction = f'''
You are a food menu analysis assistant.
Task:
Decompose each restaurant menu item into its likely ingredients.

Input:
You will be given a list of menu items. Each item contains:
- item_id
- item_name
- restaurant_id
- optional description

Output requirements (STRICT):
- Output MUST be machine-readable.
- Output MUST follow the exact numbered format shown below.
- Do NOT include extra commentary, explanations, or markdown.
- Do NOT invent fields that are not requested.
- Use only plain text.
- Each ingredient must be a short noun phrase.
- Ingredients must be ordered from most prominent to least prominent.

For each menu item, infer:
1) The most likely ingredients commonly used in this dish
2) The reasoning explaining WHY those ingredients are present, based on:
   - culinary conventions
   - dish name semantics
   - regional cooking practices
   - typical restaurant preparation methods

Constraints:
- If an ingredient is uncertain, still include it but reflect uncertainty in the reasoning.
- Do NOT include cooking utensils, heat, or generic terms like seasoning(salt, pepper, etc) unless necessary.
- Avoid overly granular items (e.g., “Himalayan pink salt” → “salt”).
'''

gemini_response = gemini_client.responses.create(
    model=gemini_model,
    input=gemini_menu_items,
    instructions=gemini_instruction,
    text=gemini_json_schema,
)

# print(gemini_response.output_text)

In [ ]:
print(gemini_response)

Response(id='resp_SBydTsmkdrDQLsZbUrtvKkLO1WNjUwsYK0Yej3hZMokJqoeEc14h0ieagIw6A9Mz5AMuDAFSaHJxBHygMKGz7rV-31r63x7h3eXBKjNZY21RCk8wFUqZcwtM4zoCn4XH6TmwzPjwoFh1bkxh0Y8BFQWjU4M5HQM_eJGPSS_gdD3428410hA2k143QT-pvhRtnXTU91i-VCOiBrMBmnTT2t5wivmf3zjkI7pXOo2ktPC1UfFEw1Exr0LVlkjFUQZDAPCcuI5Un969qolSZjh3lCG_WOmDkyevPtKe4iVzcBclgy3Nl4KZfJKI2xhRCrOA8XZujnj8pFpWlps3N06aiyADK9nwFNVwly883FA24oSmIjFd3O3ypmElEtvgmXwmNAJVnA8xtd2hn5W4WxXuCEuc3-shEL8rYpFiyHu-SgRGPqeaTRjReTxXS-VL9KZyORzXgrCJ7Mp6JOR1NG-npzc6bHKgzUiJNTHc-U2gRTVks6o=', created_at=1769592156.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='gemini-2.5-pro', object='chat.completion', output=[ResponseOutputMessage(id='XdV5abCoHMi8gLUP8euNoAk', content=[ResponseOutputText(annotations=[], text='{\n  "results": [\n    {\n      "item_id": "1",\n      "item_name": "Extra Large Meat Lovers Whole Pie",\n      "restaurant_id": "1",\n      "ingredients": [\n        "Pizza dough",\n        "Tomato sauce",\n        "Mozzarella c

In [ ]:
gemini_data = json.loads(gemini_response.output_text)
print(type(gemini_data))
print(gemini_data["results"][1])  # individual item result

<class 'dict'>
{'item_id': '10', 'item_name': 'Jumbo Chicken Wings Five Pieces', 'restaurant_id': '1', 'ingredients': ['Chicken wings', 'Flour coating', 'Wing sauce', 'Vegetable oil'], 'reasoning': "The primary ingredient is 'Chicken wings' as specified in the item name. Restaurant-style wings are typically coated in flour for a crispy texture when fried in vegetable oil. After frying, they are tossed in a 'wing sauce,' which could be Buffalo, BBQ, or another variety."}


In [ ]:
# Claude (claude-4-sonnet)
claude_api_key = "sk-7nZuV2mGMDXZRxETx6rk6w"

claude_base_url = "https://aiapi-prod.stanford.edu/v1"
claude_model = "claude-4-sonnet"

claude_client = OpenAI(api_key=claude_api_key, base_url=claude_base_url)
claude_menu_items = "1 item_id: 1; restaurant_id: 1; Extra Large Meat Lovers Whole Pie 2 item_id: 10; restaurant_id: 1; Jumbo Chicken Wings Five Pieces 3 item_id: 19; restaurant_id: 1; Bon Appetit Cheese and Berry Danish Single"
claude_json_schema = {
    "format": {
        "type": "json_schema",
        "name": "results",
        "strict": True,
        "schema": {
            "type": "object",
            "additionalProperties": False,
            "properties": {
                "results": {
                    "type": "array",
                    "items": {
                        "type": "object",
                        "additionalProperties": False,
                        "properties": {
                            "item_id": {"type": "string"},
                            "item_name": {"type": "string"},
                            "restaurant_id": {"type": "string"},
                            "ingredients": {
                                "type": "array",
                                "items": {
                                    "type": "object",
                                    "additionalProperties": False,
                                    "properties": {
                                        "name": {"type": "string"},
                                        "confidence": {
                                            "type": "number",
                                            "minimum": 0.0,
                                            "maximum": 1.0
                                        }
                                    },
                                    "required": ["name", "confidence"]
                                }
                            },
                            "reasoning": {"type": "string"}
                        },
                        "required": [
                            "item_id",
                            "item_name",
                            "restaurant_id",
                            "ingredients",
                            "reasoning"
                        ]
                    }
                }
            },
            "required": ["results"]
        }
    }
}

claude_instruction = f'''
You are a food menu analysis assistant.
Task:
Decompose each restaurant menu item into its likely ingredients.

Input:
You will be given a list of menu items. Each item contains:
- item_id
- item_name
- restaurant_id
- optional description

Output requirements (STRICT):
- Output MUST be machine-readable.
- Output MUST follow the exact numbered format shown below.
- Do NOT include extra commentary, explanations, or markdown.
- Do NOT invent fields that are not requested.
- Use only plain text.
- Each ingredient must be a short noun phrase.
- Ingredients must be ordered from most prominent to least prominent.

For each menu item, infer:
1) The most likely ingredients commonly used in this dish
2) The reasoning explaining WHY those ingredients are present, based on:
   - culinary conventions
   - dish name semantics
   - regional cooking practices
   - typical restaurant preparation methods

Constraints:
- If an ingredient is uncertain, still include it but reflect uncertainty in the reasoning.
- Do NOT include cooking utensils, heat, or generic terms like seasoning(salt, pepper, etc) unless necessary.
- Avoid overly granular items (e.g., “Himalayan pink salt” → “salt”).
'''

claude_response = claude_client.responses.create(
    model=claude_model,
    input=claude_menu_items,
    instructions=claude_instruction,
    text=claude_json_schema,
)

In [ ]:
print(claude_response)

Response(id='resp_kcf-juCg2Wt-agkZF_sG8CgVkx-RY9JniDOhOiSZWMXGPYLgg7dcdZIQX7r0_4k8oM_pdqcgnAYRB4GUMNmXhxftDEPtwkqamTBUNvHydWTvpMxCmysgyX73k3ONyqkEi5vLpdne67irOZIZ9Hu812EZCfVDR9JYoZmUUru6GnA6iX5Q6LZ0Ru8nerji3ELc_gKOKy1sXwHGPJahXfn_o8h9FIv_Wt0dr3iy61XhN9mEKpjOB8UikFKvwWy_ZO3bPbaS2k2FrDi8gfjkyi5l2bgGVtrZwGl5eiXHMfFnOQwOMYcw2XxFTHb5XtqA9HddzLI_NTuK8cR25SNaD66s7MbJJTEA7s-6jcFV_2gPHyFcoDkPeRXm-8qvOPbueylWfUTRWdjdXRi9xTlu8DEeCPWb7ZIj6Q6VD6Blq1R-XBJ6P4soLdvAVdCBe8BcrlMwHlvAZ2QIpC7Hpk-N2quzP9Jte4LsONCtCo3SLDfBGlbUitsy5w0LQ61BaTB-8uDxJHF1hxOR1HeWPJmDkCFv79x63A==', created_at=1769593615.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='claude-sonnet-4@20250514', object='chat.completion', output=[ResponseOutputMessage(id='chatcmpl-70f5f58a-70a1-4202-9c34-537b98591e56', content=[ResponseOutputText(annotations=[], text='{"results": [{"item_id": "1", "item_name": "Extra Large Meat Lovers Whole Pie", "restaurant_id": "1", "ingredients": ["pizza dough", "tomato sauce", "moz

In [ ]:
claude_data = json.loads(claude_response.output_text)
print(type(claude_data))
print(claude_data["results"][1])  # individual item result

<class 'dict'>
{'item_id': '10', 'item_name': 'Jumbo Chicken Wings Five Pieces', 'restaurant_id': '1', 'ingredients': [{'name': 'chicken wings', 'confidence': 0.98}, {'name': 'flour', 'confidence': 0.75}, {'name': 'hot sauce', 'confidence': 0.7}, {'name': 'butter', 'confidence': 0.65}, {'name': 'garlic powder', 'confidence': 0.6}, {'name': 'paprika', 'confidence': 0.55}], 'reasoning': 'Jumbo chicken wings are typically prepared by coating raw chicken wings in seasoned flour before frying or baking. Most restaurant wings are served with a buffalo-style sauce made from hot sauce and butter. Common seasonings include garlic powder and paprika for flavor enhancement and color.'}


In [ ]:
restaurant_name = "Jinsei Sushi"
restaurant_id = 7


In [ ]:
cur.execute("select * from menus where restaurant_id = 7;")
a = cur.fetchall()
print(a)

[(7, 'Picked for you', 'Red Dragon ', 'Spicy tuna tartare, tuna tataki, cucumber, scallions, sesame dressing. ', '16.0 USD'), (7, 'Picked for you', 'Edamame', 'Steamed or grilled. Served with ponzu.', '8.0 USD'), (7, 'Picked for you', 'Spicy Hotate', 'Spicy scallop, avocado, crab salad, tobiko, tempura flakes, eel sauce, sriracha.', '17.0 USD'), (7, 'Picked for you', 'Kadoma Tuna', 'Spicy tuna tartare, tempura rice cake, avocado, jalapeno, tobiko, sesame seeds, eel sauce.', '17.0 USD'), (7, 'Picked for you', 'Spiro Roll', 'Spicy tuna tartare, sake, tempura asparagus, avocado, spicy aioli, chili powder.', '16.0 USD'), (7, 'Starters', 'Miso Soup', None, '8.0 USD'), (7, 'Starters', 'Chili Miso Soup', None, '8.0 USD'), (7, 'Starters', 'Grilled Shishito Peppers', 'Served With Ponzu.', '9.0 USD'), (7, 'Starters', 'Edamame', 'Steamed or grilled. Served with ponzu.', '8.0 USD'), (7, 'Starters', 'Green Beans', 'Choose between tempura fried or grilled green beans.', '8.0 USD'), (7, 'Starters', '

(7, 'Picked for you', 'Red Dragon ', 'Spicy tuna tartare, tuna tataki, cucumber, scallions, sesame dressing. ', '16.0 USD')


In [ ]:
chatgpt_base_url = "https://aiapi-prod.stanford.edu/v1"
chatgpt_model = "gpt-5"

chatgpt_api_version = "2025-03-01-preview"

chatgpt_client = OpenAI(
    api_key="sk-7nZuV2mGMDXZRxETx6rk6w",
    base_url=chatgpt_base_url,
    default_query={"api-version": chatgpt_api_version},
)
sustainability_schema = {
    "format": {
        "type": "json_schema",
        "name": "restaurant_sustainability",
        "strict": True,
        "schema": {
            "type": "object",
            "additionalProperties": False,
            "properties": {
                "restaurant": {"type": "string"},
                "sustainability_score": {
                    "type": "integer",
                    "minimum": 1,
                    "maximum": 5
                },
                "reasoning": {
                    "type": "object",
                    "additionalProperties": False,
                    "properties": {
                        "plant_based_ratio": {"type": "string"},
                        "red_meat_impact": {"type": "string"},
                        "high_impact_ingredients": {"type": "string"},
                        "processing_level": {"type": "string"},
                        "overall_explanation": {"type": "string"}
                    },
                    "required": [
                        "plant_based_ratio",
                        "red_meat_impact",
                        "high_impact_ingredients",
                        "processing_level",
                        "overall_explanation"
                    ]
                }
            },
            "required": ["restaurant", "sustainability_score", "reasoning"]
        }
    }
}

sus_instruction = f'''
You are a food sustainability analyst.

You will be given:
- A restaurant name
- Its location
- A list of menu items
- Each menu item has a list of ingredients

Your task is to assign a sustainability score from 1 to 5, where:

1 = very high environmental impact (beef-heavy, fried, ultra-processed)
3 = mixed or average impact
5 = very sustainable (plant-based, low red meat, fresh, low carbon footprint)

Base your score ONLY on the ingredients provided.

Consider these factors:
- Percentage of plant-based and vegetarian items
- Amount of beef, lamb, cheese, and butter
- Use of poultry or seafood instead of red meat
- Presence of vegetables, legumes, and whole grains
- Use of highly processed or fried foods
- Use of imported or high-footprint ingredients (e.g., shrimp, avocado, dairy, beef)

Restaurant:
Name: Jinsei Sushi
Location: 1830 29th Ave S, Birmingham, AL, 35209

Output requirements (STRICT):
- Output MUST be machine-readable.
- Do NOT include extra commentary, explanations, or markdown.
- Do NOT invent fields that are not requested.
- Use only plain text.

'''


chatgpt_response = chatgpt_client.responses.create(
    model=chatgpt_model,
    input=str(items),
    instructions=sus_instruction,
    text=sustainability_schema,
)

In [ ]:
sus_data = json.loads(chatgpt_response.output_text)
print(sus_data)

{'restaurant': 'Jinsei Sushi', 'sustainability_score': 2, 'reasoning': {'plant_based_ratio': 'Low; a few sides/salads (edamame, shishito peppers, seaweed/kelp salads, baby greens) are plant-based, but most items center on seafood or meat.', 'red_meat_impact': 'Not frequent but significant: Kobe beef (sliders, wrapped asparagus, buns), beef in a roll, and foie gras add high impact despite being a minority.', 'high_impact_ingredients': 'Shrimp (tempura, rock shrimp), lobster, unagi (eel), tuna/yellowtail, octopus, roe, avocado, cream/goat cheese, mayonnaise/aioli; some imitation crab.', 'processing_level': 'Mixed to high: many tempura-fried items and rich sauces (eel sauce, spicy mayo/aioli), alongside minimally processed sashimi/ceviche dishes.', 'overall_explanation': 'Seafood-dominant menu with frequent tempura frying and rich sauces, limited vegetarian options, and notable high-impact items (shrimp, lobster, unagi, Kobe beef, foie gras). Overall footprint below average sustainability